# Bedingungserfüllungsprobleme
- Englisch: Constraint-Satisfaction-Problems [Kurz: CSPs]
- bestehen aus:
    - Variablen
    - Domänen (Wertebereiche für Variablen)
    - Bedingungen zwischen Variablen
        - müssen erfüllt sein um das Problem zu lösen
- Beispiel:
    - Freitagsmeeting für 3 Personen planen
    - 3 Personen als Variablen
        - Joe
        - Mary
        - Sue
    - Die Domäne ist die Verfügbarkeit der 3 Personen
        - z.B.: für Mary: 14, 15, 16 Uhr
    - 2 Bedingungen:
        1. Sue muss teilnehmen
        2. mind. 2 Personen müssen teilnehmen
---

## Framework für Bedingungserfüllungsprobleme
- Bedingungen werden mit einer Constraint Klasse definiert
- Jede Bedingung besteht aus den Variablen für die sie Bedingungen setzen
- Methode "erfüllt" überprüft ob die Bedingung erfüllt ist
- Hauptlogik: 
    - Prüfung ob eine Bedingung erfüllt ist

In [1]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # Variablentyp
D = TypeVar('D') # Domänentyp

# Basisklasse für alle Bedingungen
class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables
    # muss von Unterklassen überschrieben werden!
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        pass

- Abstrakte Basisklasseen dienen als Vorlagen für Klassenhierarchie
- nur für Frameworks nicht für Klassenhirarchie gedacht in Python!
- Backtracking Algorythmus für die Lösung für Probleme zu finden
    - Immer wenn ein Ergebnis nicht das Ziel ist wird zun letzten Entscheidungspunkt zurück gegeangen
    - Ähnlich der Tiefensuche
    - Rekursive Tiefensuche

In [2]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables
        self.domains: Dict[V, List[D]] = domains
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Jeder Variablen sollte einer Domäne zugewiesen sein.")
    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Jeder Variable in Bedingung nicht in CSP.")
            else:
                self.constraints[variable].append(constraint)
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True
    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # Zuordnung vollständig, wenn jede Viariable zugeordnet ist (Abbruchbedingung)
        if len(assignment) == len(self.variables):
            return assignment
        # Alle Variablen holen, die im CSP, aber in keiner Zuordnung sind
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
        # Jedem möglichen Domänenwert der ersten nicht zugeordneten Variable holen
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # Wenn wir noch konsistent sind, Rekursion (fortfahren)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # Wenn wir das Ergebnis nicht gefunden haben, backtracking
                if result is not None:
                    return result
        return None